# Generacion de Datasets utilizando RLHF para fine tunning

## Procedimiento

- Recoleccion de Datos: Obtener Informacion mediante documentos de textos seleccionables (no escaneados), mediante web, redes sociales, scrapping,etc
- Extraer Texto: Usa PyMuPDF para extraer texto de PDFs no escaneados, procesando en paralelo con ProcessPoolExecutor para manejar grandes volúmenes (>100, >10,000).
- Limpiar Datos: Elimina espacios múltiples y caracteres no deseados con expresiones regulares, asegurando texto coherente.
- Validar: Verifica que los fragmentos extraídos no estén vacíos y tengan una longitud mínima (por ejemplo, 50 caracteres).
- Generar Conversaciones: Divide el texto en párrafos, usa un modelo como meta-llama/Llama-3.2-1B-Instruct para generar diálogos conversacionales específicos al contenido, con al menos 4 intercambios por conversación, en formato {"messages": [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}], "topic": "..."}.
- Estructurar Dataset: Guarda las conversaciones en archivos JSONL con dos columnas: "messages" y "topic".
Cargar y Subir: Carga el dataset con load_dataset("json", data_files="path/*.jsonl") y súbelo a un repositorio privado en Hugging Face Hub con push_to_hub("username/dataset_name", private=True).

In [2]:
%pip install --upgrade pymupdf nltk ollama torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 27.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 20.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 29.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 240.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 129.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 197.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 25.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 M

In [1]:
import torch

torch.cuda.is_available()

/workspace/fine-tunning-models/.venv/lib/python3.11/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


True

In [2]:
import os

OLLAMA_MODEL = "llama3.1:8b"
TEMPERATURE = 0.7  # Equilibrio entre creatividad y coherencia
TOP_P = 0.9  # Filtrado de núcleo para diversidad
MIN_CONVERSATION_LENGTH = 3 

# Constantes
MIN_FRAGMENT_LENGTH = 500
MAX_FRAGMENT_LENGTH = 2000
REPEAT_THRESHOLD = 0.3
MIN_BLOCK_LENGTH = 30  # Reducido de 50 a 30 para incluir bloques más cortos

### Recoleccion de Datos

In [3]:
from pathlib import Path

# Verificar si la carpeta existe

folder_url = "./docs"
folder = Path(folder_url)


if folder.exists() and folder.is_dir():
    print("Valid Folder")
    
# Obtener todos los archivos de la carpeta
files = [f for f in folder.rglob("*") if f.is_file()]

files
len(files)

Valid Folder


89

### Extraccion de Texto y Limpieza de Datos

In [4]:
import pymupdf
import re
import os
import hashlib
from collections import Counter



def clean_text(text, repeated_blocks=None):
    """Limpia el texto, eliminando caracteres repetitivos, texto redundante y caracteres no deseados."""
    # Eliminar caracteres de control no deseados (excepto \n)
    text = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]', '', text)
    # Eliminar patrones repetitivos como ----, ...., ****
    text = re.sub(r'([^\w\s])\1{2,}|\s*[.]{3,}\s*', '', text)
    # Normalizar espacios múltiples
    text = re.sub(r'[ \t]+', ' ', text)
    # Eliminar espacios al inicio y final de cada línea
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    # Filtrar líneas duplicadas, números, correos y contenido administrativo
    seen_lines = set()
    unique_lines = []
    for line in lines:
        if line not in seen_lines and \
           not re.match(r'^\d+$', line) and \
           not re.match(r'.*@(.*\.)+.*', line) and \
           not re.match(r'^(Tel|Fax|E-mail|www\.).*', line, re.IGNORECASE) and \
           (repeated_blocks is None or line not in repeated_blocks):
            unique_lines.append(line)
            seen_lines.add(line)
    return '\n'.join(unique_lines)

def extract_text_from_pdf(pdf_path):
    """
    Extrae texto de un PDF en una sola pasada, preservando el texto del inicio de la página,
    eliminando encabezados, pies de página y contenido irrelevante,
    dividiendo en fragmentos de 500 a 2000 caracteres con metadata corregida.
    """
    try:
        doc = pymupdf.open(pdf_path)
        total_pages = len(doc)
        chunks = []
        current_chunk = []  # Lista de (párrafo, página)
        current_chunk_length = 0
        filename = os.path.basename(pdf_path)
        block_counter = Counter()

        for page_number in range(1, total_pages + 1):
            page = doc[page_number - 1]
            page_height = page.rect.height
            blocks = page.get_text("blocks")
            page_text = []

            # Procesar bloques y filtrar encabezados/pies de página
            for block in blocks:
                text = block[4]
                y0, y1 = block[1], block[3]
                # Excluir pies de página (parte inferior de la página)
                if y1 > 0.95 * page_height:  # Relajado de 0.95 a 0.9
                    continue
                # Excluir encabezados solo si son repetitivos
                if y0 < 0.05 * page_height:  # Relajado de 0.05 a 0.1
                    block_counter[text] += 1
                    if block_counter[text] > total_pages * REPEAT_THRESHOLD:
                        continue
                if text and len(text) >= MIN_BLOCK_LENGTH:
                    block_counter[text] += 1
                    page_text.append((text, page_number))

            # Si no hay texto válido en la página, continuar
            if not page_text:
                continue

            # Acumular párrafos con su número de página
            for paragraph, page in page_text:
                current_chunk.append((paragraph, page))
                current_chunk_length += len(paragraph) + 2  # +2 por "\n\n"

                # Si el fragmento alcanza la longitud mínima, procesarlo
                if current_chunk_length >= MIN_FRAGMENT_LENGTH:
                    chunk_text = "\n\n".join(p for p, _ in current_chunk)
                    cleaned_chunk = clean_text(chunk_text, None)
                    cleaned_length = len(cleaned_chunk)

                    # Obtener el rango de páginas del fragmento
                    page_numbers = sorted(set(page for _, page in current_chunk))
                    start_page = page_numbers[0] if page_numbers else page_number
                    end_page = page_numbers[-1] if page_numbers else page_number

                    # Dividir fragmentos largos
                    while cleaned_length > MAX_FRAGMENT_LENGTH:
                        sub_chunk = cleaned_chunk[:MAX_FRAGMENT_LENGTH]
                        last_paragraph_end = sub_chunk.rfind("\n\n")
                        if last_paragraph_end == -1:
                            last_paragraph_end = MAX_FRAGMENT_LENGTH
                        chunk_to_add = cleaned_chunk[:last_paragraph_end].strip()

                        # Calcular el número de páginas para el subfragmento
                        chars_so_far = 0
                        sub_chunk_pages = []
                        for paragraph, page in current_chunk:
                            chars_so_far += len(paragraph) + 2
                            if chars_so_far <= last_paragraph_end:
                                sub_chunk_pages.append(page)
                            else:
                                break
                        sub_start_page = min(sub_chunk_pages) if sub_chunk_pages else page_number
                        sub_end_page = max(sub_chunk_pages) if sub_chunk_pages else page_number

                        metadata = (
                            f"# FILENAME: {filename} | CHARACTERS: {len(chunk_to_add)} | "
                            f"PAGES: {sub_start_page}-{sub_end_page}/{total_pages}\n\n"
                        )
                        chunks.append((metadata + chunk_to_add, hashlib.md5(chunk_to_add.encode()).hexdigest()))
                        cleaned_chunk = cleaned_chunk[last_paragraph_end:].strip()
                        cleaned_length = len(cleaned_chunk)

                        # Actualizar current_chunk para los párrafos restantes
                        remaining_chunk = []
                        chars_so_far = 0
                        for paragraph, page in current_chunk:
                            chars_so_far += len(paragraph) + 2
                            if chars_so_far > last_paragraph_end:
                                remaining_chunk.append((paragraph, page))
                        current_chunk = remaining_chunk
                        current_chunk_length = cleaned_length
                        page_numbers = sorted(set(page for _, page in current_chunk))
                        start_page = page_numbers[0] if page_numbers else page_number

                    # Añadir el fragmento completo
                    if cleaned_length >= MIN_FRAGMENT_LENGTH:
                        metadata = (
                            f"# FILENAME: {filename} | CHARACTERS: {cleaned_length} | "
                            f"PAGES: {start_page}-{end_page}/{total_pages}\n\n"
                        )
                        chunks.append((metadata + cleaned_chunk, hashlib.md5(cleaned_chunk.encode()).hexdigest()))
                        current_chunk = []
                        current_chunk_length = 0

                    else:
                        current_chunk = [(cleaned_chunk, page_numbers[-1])] if page_numbers else []
                        current_chunk_length = cleaned_length

        # Añadir el fragmento final si cumple con la longitud mínima
        if current_chunk and current_chunk_length >= MIN_FRAGMENT_LENGTH:
            chunk_text = "\n\n".join(p for p, _ in current_chunk)
            cleaned_chunk = clean_text(chunk_text, None)
            cleaned_length = len(cleaned_chunk)
            if cleaned_length >= MIN_FRAGMENT_LENGTH:
                page_numbers = sorted(set(page for _, page in current_chunk))
                start_page = page_numbers[0] if page_numbers else total_pages
                end_page = page_numbers[-1] if page_numbers else total_pages
                metadata = (
                    f"# FILENAME: {filename} | CHARACTERS: {cleaned_length} | "
                    f"PAGES: {start_page}-{end_page}/{total_pages}\n\n"
                )
                chunks.append((metadata + cleaned_chunk, hashlib.md5(cleaned_chunk.encode()).hexdigest()))

        doc.close()

        # Filtrar bloques repetitivos y duplicados
        repeated_blocks = {text for text, count in block_counter.items() if count > total_pages * REPEAT_THRESHOLD}
        final_chunks = []
        seen_hashes = set()

        for chunk, chunk_hash in chunks:
            chunk_text = '\n'.join(line for line in chunk.splitlines() if not line.startswith('#'))
            cleaned_chunk = clean_text(chunk_text, repeated_blocks)
            if len(cleaned_chunk) >= MIN_FRAGMENT_LENGTH and chunk_hash not in seen_hashes:
                # Actualizar la longitud en la metadata después de la limpieza final
                metadata_lines = chunk.splitlines()[0]
                metadata = re.sub(r'CHARACTERS: \d+', f'CHARACTERS: {len(cleaned_chunk)}', metadata_lines)
                final_chunks.append(f"{metadata}\n\n{cleaned_chunk}")
                seen_hashes.add(chunk_hash)

        return final_chunks if final_chunks else None
    except Exception as e:
        print(f"Error procesando {pdf_path}: {e}")
        return None

### Generar Conversaciones

In [5]:
from ollama import AsyncClient
from pydantic import BaseModel, Field
from typing import List


# Definir el esquema Pydantic para la salida estructurada
class Message(BaseModel):
    role: str = Field(..., pattern="^(system|user|assistant)$")
    content: str

class Conversation(BaseModel):
    messages: List[Message] = Field(..., min_items=MIN_CONVERSATION_LENGTH)
    topic: str

ollama_client = AsyncClient()

def get_prompt(fragment:str):
  return f"""
Basado en el siguiente contexto:
<context>
{fragment}
</context>

<instructions>
Genera una conversación entre un usuario y un asistente basada **exclusivamente** en el contenido de <context>...</context>. La conversación debe seguir este formato y estilo:

1. **Formato de salida**:
   - Devuelve un objeto JSON con dos campos:
     - "messages": lista de objetos con la forma {{ "role": "user" | "assistant", "content": "string" }}
     - "topic": cadena corta (máximo 50 caracteres) que resume el tema del fragmento.

2. **Estructura de la conversación**:
   - Longitud: entre **1 y 2 intercambios** (2 a 4 mensajes).
   - Usa UNA de estas dos opciones:
     - **Opción A**: 1 intercambio (2 mensajes). Ambos deben ser **largos y detallados**.
     - **Opción B**: 2 intercambios (4 mensajes). Cada respuesta debe ser extensa y cubrir bien el contenido.
   - cada intercambio (par de mensajes) debe cumplir lo siguiente: empieza con el role: "user" y termina con el role: "assistant", en otras palabras, formato conversacional cumpliento el formato SFT (Supervices Fine Tunning)

3. **Intervenciones del usuario**:
   - **No incluyas frases como**:
     - “según el texto”, “en el fragmento”, “respecto al documento”, “basado en este artículo”, etc.
   - Deben ser naturales, humanas y relevantes.
   - Pueden adoptar estas formas:
     - Preguntas específicas o inferenciales.
     - Frases incompletas para completar.
     - Comentarios o afirmaciones parcialmente desarrolladas.
   - Las intervenciones deben seguir un flujo lógico y coherente.

4. **Respuestas del asistente**:
   - Claras, profesionales y bien estructuradas.
   - Basadas **únicamente en el contenido del contexto**.
   - Extensas, ricas en detalles y sin añadir información inventada.
   - Usa un tono conversacional técnico o divulgativo según el tema.

5. **Cobertura y contenido**:
   - Cubre **todo el contenido** del fragmento.
   - No ignores secciones importantes ni temas mencionados.
   - Si el texto es poco claro, genera igualmente un solo intercambio lo más plausible posible.

6. **Objetivo de esta conversación**:
   - Este formato será usado para entrenar modelos mediante *Supervised Fine-Tuning (SFT)* usando `SFTTrainer` de Hugging Face.
   - Busca generar conversaciones verosímiles y útiles para entrenar modelos en comprensión lectora, generación coherente y natural.

<example>
**Fragmento técnico**:
PERIODO DE TRANSICIÓN RELATIVO A LA IMPLEMENTACIÓN GENERAL DEL ACUERDO  
215. El Nuevo Acuerdo será aplicable a todos los bancos internacionalmente activos en cada nivel del grupo bancario. Aquellos países en los que la subconsolidación no es actualmente un requisito, tendrán un periodo de transición de tres años, a partir de la fecha de implementación, para aplicarla.  

PERIODO DE TRANSICIÓN RELATIVO AL MÉTODO FUNDADO EN LA CALIFICACIÓN INTERNA  
216. El Comité reconoce que un respeto completo e inmediato de ciertos requisitos relacionados con datos podría no ser posible, particularmente para exposiciones frente a empresas, bancos, soberanos y retail. Para estos casos, se prevé un periodo de transición limitado durante el cual las entidades podrán utilizar aproximaciones alternativas.

**Salida esperada**:
```json
{{
  "messages": [
    {{
      "role": "user",
      "content": "¿Qué ocurre en los países donde la subconsolidación no es obligatoria y cómo afecta esto a la aplicación del acuerdo?"
    }},
    {{
      "role": "assistant",
      "content": "En esos países, se establece un periodo de transición de tres años a partir de la implementación del acuerdo. Esto les permite adaptarse gradualmente a los nuevos requisitos regulatorios sin enfrentar impactos abruptos. La subconsolidación es esencial para asegurar una visión completa del riesgo dentro de los grupos bancarios, y este periodo transitorio facilita su adopción progresiva."
    }},
    {{
      "role": "user",
      "content": "¿Qué flexibilidades se contemplan para las entidades financieras en relación con los datos requeridos por el método basado en calificación interna?"
    }},
    {{
      "role": "assistant",
      "content": "El texto reconoce que ciertos requisitos de datos pueden ser difíciles de cumplir de inmediato, especialmente en exposiciones a empresas, bancos, gobiernos y clientes minoristas. Por ello, se permite un periodo de transición limitado durante el cual las instituciones pueden aplicar aproximaciones alternativas. Esto proporciona margen para ajustar los sistemas internos y recopilar datos adecuados sin comprometer la supervisión del riesgo."
    }},
    {{
      "role": "user",
      "content":"acuerdo 215"
    }},
    {{
      "role": "assistant",
      "content":"El Nuevo Acuerdo se aplicará a todos los bancos con actividad internacional en cada nivel de su grupo bancario. Los países donde la subconsolidación no sea actualmente obligatoria dispondrán de un período de transición de tres años, a partir de la fecha de implementación, para adoptarla."
    }}
  ],
  "topic": "Transición en la implementación del acuerdo"
}}
```

**Fragmento simple**:
El cielo es azul porque las moléculas en la atmósfera dispersan la luz azul del sol en todas direcciones. Esta dispersión se llama dispersión de Rayleigh.

**Salida esperada**:
```json
{{
  "messages": [
    {{
      "role": "user",
      "content": "¿Por qué el cielo tiene ese color azul tan característico durante el día?"
    }},
    {{
      "role": "assistant",
      "content": "El color azul del cielo se debe a un fenómeno llamado dispersión de Rayleigh. Cuando la luz del sol entra en la atmósfera terrestre, interactúa con las moléculas de aire. La luz azul, que tiene una longitud de onda más corta, se dispersa mucho más fácilmente que otros colores. Esta dispersión hace que el azul se difunda en todas direcciones, lo cual es lo que vemos desde cualquier punto del planeta durante el día. Por eso el cielo parece predominantemente azul cuando el sol está alto."
    }}
  ],
  "topic": "Dispersión de la luz en la atmósfera"
}}
```
</example>
    """


async def generate_conversation(fragment, temperature=TEMPERATURE, top_p=TOP_P):
    """
    Genera una conversación estructurada en formato JSON usando Ollama, basada en un fragmento de texto.
    Optimizado para múltiples iteraciones en la generación de datasets para fine-tuning.
    """
    if not fragment or len(fragment) < MIN_FRAGMENT_LENGTH:
        print(f"Error: Fragmento demasiado corto ({len(fragment)} caracteres).")
        return None

    # Extraer el texto sin la metadata (líneas que comienzan con '#')
    fragment_content = '\n'.join(line for line in fragment.splitlines() if not line.startswith('#')).strip()
    if len(fragment_content) < MIN_FRAGMENT_LENGTH:
        print(f"Error: Contenido útil del fragmento demasiado corto ({len(fragment_content)} caracteres).")
        return None

    # Prompt optimizado para múltiples iteraciones
    prompt = get_prompt(fragment_content)

    try:
        # Llama a Ollama con roles system y user
        response = await ollama_client.chat(
            model=OLLAMA_MODEL,
            messages=[
                {
                    "role": "system",
                    "content": "Eres un experto en el dominio del fragmento proporcionado (por ejemplo, regulaciones financieras, riesgos financieros, finanzas). Genera respuestas claras, concisas y basadas únicamente en el fragmento, manteniendo un tono profesional y conversacional, se especifico y detalla una respuestas en el intercambio de asistente."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            options={
                "temperature": temperature,
                "top_p": top_p,
                #"num_ctx":
                
                
            },
            format=Conversation.model_json_schema()  # Especifica el esquema JSON
            
        )
        conversation = Conversation.model_validate_json(response.message.content)
        # Validar que la conversación tenga al menos un intercambio completo
        if len(conversation.messages) < 2:
            print(f"Error: Conversación inválida, número de mensajes insuficiente: {len(conversation.messages)}")
            return None
        # Si el número de mensajes es impar, eliminar el último (asumiendo que es del usuario)
        if len(conversation.messages) % 2 != 0 and len(conversation.messages) > 2:
            conversation.messages = conversation.messages[:-1]
        return conversation
    except Exception as e:
        print(f"Error generando conversación con Ollama: {e}")
        return None

In [57]:
import json
import os
from tqdm.asyncio import tqdm_asyncio

class MetadataManager:
    """Clase para manejar archivos de metadatos JSON."""
    def __init__(self, index: int, output_dir: str):
        self.folder = os.path.join(output_dir, "metadata")
        self.metadata_file = os.path.join(self.folder, f"metadata_{index:04d}.json")
        os.makedirs(self.folder, exist_ok=True)

    def exists(self) -> bool:
        """Verifica si el archivo de metadatos existe."""
        return os.path.exists(self.metadata_file)

    def get(self, param: str):
        """Obtiene chunks, conversations o fileName desde el archivo de metadatos."""
        if not self.exists():
            return [] if param in ["chunks", "conversations"] else ""
        try:
            with open(self.metadata_file, "r", encoding="utf-8") as f:
                metadata = json.load(f)
                return metadata.get(param, []) if param in ["chunks", "conversations"] else metadata.get(param, "")
        except Exception as e:
            print(f"Error al leer {param} desde {self.metadata_file}: {e}")
            return [] if param in ["chunks", "conversations"] else ""

    def set(self, param: str, value):
        """Establece chunks, conversations o fileName en el archivo de metadatos."""
        try:
            metadata = {"chunks": [], "fileName": "", "conversations": []}
            if self.exists():
                with open(self.metadata_file, "r", encoding="utf-8") as f:
                    metadata = json.load(f)
            
            metadata[param] = value
            with open(self.metadata_file, "w", encoding="utf-8") as f:
                json.dump(metadata, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"Error al escribir {param} en {self.metadata_file}: {e}")

def get_text_from_pdf(index: int, pdf_path: str, output_dir: str) -> list:
    """Obtiene el texto de un PDF desde la caché (JSON) o lo extrae si no existe."""
    metadata_manager = MetadataManager(index, output_dir)
    
    if metadata_manager.exists():
        chunks = metadata_manager.get("chunks")
        file_name = metadata_manager.get("fileName")
        if chunks and file_name == os.path.basename(pdf_path):
            return chunks
    
    try:
        pages_text = extract_text_from_pdf(pdf_path)
        if pages_text:
            metadata_manager.set("chunks", pages_text)
            metadata_manager.set("fileName", os.path.basename(pdf_path))
            metadata_manager.set("conversations", [])
        return pages_text
    except Exception as e:
        print(f"Error al extraer texto del PDF {pdf_path}: {e}")
        return []

async def get_conversation_from_chunk(index: int, output_dir: str, chunk: str):
    """Obtiene o genera una conversación para un fragmento de texto."""
    metadata_manager = MetadataManager(index, output_dir)
    existing_conversations = metadata_manager.get("conversations")
    
    for conv in existing_conversations:
        if isinstance(conv, dict) and conv.get("source_chunk") == chunk:
            return conv
    
    try:
        conversation = await generate_conversation(chunk)
        if conversation:
            conv_dict = conversation.model_dump()
            conv_dict["source_chunk"] = chunk
            return conv_dict
        return None
    except Exception as e:
        print(f"Error al generar conversación para fragmento en PDF #{index}: {e}")
        return None

async def process_pdf(index: int, pdf_path: str, output_dir: str):
    """Procesa un PDF, genera conversaciones y las guarda en JSONL."""
    try:
        pages_text = get_text_from_pdf(index, pdf_path, output_dir)
        
        if not pages_text:
            print(f"No se encontraron fragmentos de texto para el PDF #{index}: {pdf_path}")
            return
        
        data_dir = os.path.join(output_dir, "data")
        os.makedirs(data_dir, exist_ok=True)
        jsonl_file = os.path.join(data_dir, f"pdf_{index:04d}.jsonl")
        
        tasks = [get_conversation_from_chunk(index, output_dir, fragment) 
                 for fragment in pages_text if len(fragment) > 20]
        conversations = await tqdm_asyncio.gather(*tasks, desc=f"Procesando PDF #{index} '{pdf_path}'")
        
        valid_conversations = [conv for conv in conversations if conv is not None]
        if valid_conversations:
            # Guardar en JSONL
            with open(jsonl_file, "w", encoding="utf-8") as f:
                output_str = "\n".join(json.dumps({"messages":conv['messages'],"topic":conv['topic']}, ensure_ascii=False) for conv in valid_conversations)
                f.write(output_str + "\n")
            
            
            metadata_manager = MetadataManager(index, output_dir)
            metadata_manager.set("conversations", valid_conversations)
            metadata_manager.set("num_interactions",len(valid_conversations))
            total_messages = sum(len(conv['messages']) for conv in valid_conversations)
            metadata_manager.set("num_messages",total_messages)
        else:
            print(f"No se generaron conversaciones para el PDF #{index}: {pdf_path}")
    except Exception as e:
        print(f"Error al procesar el PDF {pdf_path}: {e}")

In [58]:
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import asyncio
import os
from pathlib import Path

def process_pdf_wrapper(args):
    index, pdf_path, output_dir = args
    # Ejecutar la función asíncrona process_pdf dentro de un bucle de eventos
    asyncio.run(process_pdf(index, pdf_path, output_dir))

def generate(max_workers=8):
    pdf_files = files
    output_dir="outputs"
    output_folder_path=Path(output_dir)
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)
    
    print(f"Generando Datasets\nmax_workers={max_workers}\n# Files: {len(pdf_files)}")
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(process_pdf_wrapper, [(i, p, output_dir) for i, p in enumerate(pdf_files)]), total=len(pdf_files)))

In [ ]:
%%time
generate(15)

Generando Datasets
max_workers=15
# Files: 89


Procesando PDF #5 'docs/Spanish-Lavado-de-Activos-Basado-en-el-Comercio-Indicadores-de-Riesgo.pdf.coredownload.inline.pdf':   0%|          | 0/12 [00:00<?, ?it/s]

No se encontraron fragmentos de texto para el PDF #13: docs/RESOLUCION UIF.pdf


Procesando PDF #5 'docs/Spanish-Lavado-de-Activos-Basado-en-el-Comercio-Indicadores-de-Riesgo.pdf.coredownload.inline.pdf': 100%|██████████| 12/12 [00:00<00:00, 336.52it/s]
Procesando PDF #15 'docs/Modelo_de_Administracion_del_Riesgo_de_LAFT_y_Contrabando_web.pdf':   0%|          | 0/592 [00:00<?, ?it/s]221 [00:00<?, ?it/s]t/s]<?, ?it/s]Task exception was never retrieved
future: <Task finished name='Task-57' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at /workspace/fine-tunning-models/.venv/lib/python3.11/site-packages/tqdm/asyncio.py:75> exception=TypeError("object NoneType can't be used in 'await' expression")>
Traceback (most recent call last):
  File "/workspace/fine-tunning-models/.venv/lib/python3.11/site-packages/tqdm/asyncio.py", line 76, in wrap_awaitable
    return i, await f
              ^^^^^^^
TypeError: object NoneType can't be used in 'await' expression
Procesando PDF #5 'docs/Spanish-Lavado-de-Activos-Basado-en-el-Comercio-Indicadores-de-Riesgo.pd

Error generando conversación con Ollama: Event loop is closed


Procesando PDF #10 'docs/Riesgo de Crédito.pdf': 100%|██████████| 11/11 [04:00<00:00, 21.82s/it]].coredownload.inline (1).pdf':  92%|█████████▏| 11/12 [03:43<00:19, 19.30s/it]
Procesando PDF #18 'docs/manual_riesgos_mercado_liquidez_9_jul_10.pdf':   0%|          | 0/56 [00:00<?, ?it/s]

Error generando conversación con Ollama: Event loop is closed


Procesando PDF #6 'docs/Spanish-Lavado-de-Activos-Basado-en-el-Comercio-Indicadores-de-Riesgo.pdf.coredownload.inline (1).pdf': 100%|██████████| 12/12 [04:39<00:00, 23.31s/it]
Procesando PDF #19 'docs/manual_operaciones_activas_contingentes_24_feb_17.pdf':   0%|          | 0/150 [00:00<?, ?it/s]

Error generando conversación con Ollama: Event loop is closed


Procesando PDF #9 'docs/RESUMEN-EJECUTIVO.pdf':  17%|█▋        | 5/30 [09:19<32:11, 77.27s/it] 1:45, 15.08s/it]  :53,  1.57s/it]

### Saving

In [11]:
from datasets import load_dataset


dataset = load_dataset("json", data_files="./outputs/*.jsonl")

dataset


Resolving data files:   0%|          | 0/96 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['messages', 'topic'],
        num_rows: 664
    })
})

In [10]:
from huggingface_hub import login
login()

ERROR! Session/line number was not unique in database. History logging moved to new session 16


In [20]:
dataset.push_to_hub("jeanmcm/b_risks")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        : 100%|##########|  270kB /  270kB            

README.md:   0%|          | 0.00/376 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jeanmcm/b_risks/commit/13e6a6ad8f5d4eb1efc3bf36fe4e0b58c6e237fb', commit_message='Upload dataset', commit_description='', oid='13e6a6ad8f5d4eb1efc3bf36fe4e0b58c6e237fb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jeanmcm/b_risks', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jeanmcm/b_risks'), pr_revision=None, pr_num=None)

# Testing RAG with Open WebUI 

In [36]:
import requests
import json
import sys
import time
from IPython.display import display, clear_output, HTML

url = "https://vwlppjjfa98c9x-8080.proxy.runpod.net"
api_key ="sk-05568562f28844fe997cadf960a346cd"

messages =  [{"role": "user", "content": "Que es el Riesgo Financiero?"}]

try:
    # Realizar la solicitud con stream=True
    with requests.post(f"{url}/api/chat/completions", stream=True,headers={
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    },json={
      "model":"bosft-riesgos-rag-model",
      "messages":messages,
      "stream":True
      }) as response:
        response.raise_for_status()
        # Variable para almacenar la salida acumulada
        accumulated_output = ""

        # Iterar sobre las líneas de la respuesta
        for line in response.iter_lines():
            if line:
                # Decodificar la línea
                decoded_line = line.decode('utf-8').strip()
                # Si la línea comienza con "data:", extraer el contenido
                if decoded_line.startswith("data:"):
                    decoded_line = decoded_line[5:].strip()  # Quitar "data: "

                # Ignorar líneas vacías o marcadores de fin como "[DONE]"
                if not decoded_line or decoded_line == "[DONE]" :
                    continue
                
                
                try:
                    # Parsear si es JSON
                    data = json.loads(decoded_line)
                    if "choices" not in data: continue
                    
                    delta = data['choices'][0]['delta']
                    if "content" in delta: new_data = delta['content']
                except json.JSONDecodeError:
                    # Si no es JSON, usar la línea como texto
                    new_data = decoded_line

                # Acumular y mostrar la salida dinámicamente
                if new_data:
                    accumulated_output += new_data
                    # Limpiar la salida anterior y mostrar la nueva
                    clear_output(wait=True)
                    display(HTML(f"<b>Respuesta en streaming:</b> {accumulated_output}"))
                    time.sleep(0.1)  # Pequeña pausa para visibilidad
                    

except requests.exceptions.RequestException as e:
    print(f"\nError en la solicitud: {e}")

# Testing with Flowise